In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Importation of useful libraries
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import seaborn as sns

import random 
import datetime as dt
import re
import pickle
import nltk, warnings
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from string import digits, punctuation


from scipy.stats import chi2_contingency

from sklearn.preprocessing import LabelEncoder, StandardScaler, Normalizer
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn import preprocessing, model_selection, metrics, feature_selection
from sklearn.model_selection import GridSearchCV, learning_curve
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn import neighbors, linear_model, svm, tree, ensemble
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from wordcloud import WordCloud, STOPWORDS

warnings.filterwarnings("ignore")
plt.style.use('bmh')
%matplotlib inline
import os
print(os.listdir("../input"))


In [ ]:
#Create the dataframe by encoding it
dataset=pd.read_csv(r'/kaggle/input/ecommerce-data/data.csv',encoding='ISO-8859-1')

#view the sample records
dataset.head()

In [ ]:
#Information about the dataset i.e column name, data types and no of non null counts
dataset.info()

In [ ]:
#Find the missing values
dataset.isna().sum()

In [ ]:
#dataset shape

rows, columns = dataset.shape
print("Dataset Info")
print("*************")
print("Number of Observations  :", rows)
print("Number of Dimensions    :", columns)


In [ ]:
#Print the column names and datatypes
dash = '-' * 70

print (dash)
print ('{:<40s}{:<2s}{:>13s}{:>3s}{:>10s}'.format('              COLUMN NAME', '|', 'UNIQUE VALUES','|','DATA TYPE'))
print (dash)
for col in dataset.columns:
    print ('{:<40s}{:<2s}{:>8s}{:>8s}{:>8s}'.format(col, '|', str(len(dataset[col].unique())), '|', str(dataset[col].dtype)))


In [ ]:
#Columns list
dataset.columns

In [ ]:
#Statistics info of dataset
dataset.describe(percentiles=[0.1, 0.25, 0.5, 0.75, 0.9])

In [ ]:
#Check if there is duplicate observations
dataset.duplicated().sum()

In [ ]:
#Drop all duplicate observations which is not needed
dataset.drop_duplicates(inplace=True)

In [ ]:
#Information without having duplicate obserations inplace
dataset.info()

In [ ]:
# Check the null values
nan_obs = dataset[dataset.isnull().T.any().T]
nan_obs

In [ ]:
#Drop the observations where customerid is null
dataset=dataset.dropna(subset=['CustomerID'])

In [ ]:
#dataset shape after removing nan records

rows, columns = dataset.shape
print("Dataset Info")
print("*************")
print("Number of Observations  :", rows)
print("Number of Dimensions    :", columns)


In [ ]:
print("Percentage(%) of the removed records : {}%".format(((541909-401604)/541909)*100))

In [ ]:
#Exploratory data analsys
#Number contries present in the dataset
dataset['Country'].nunique()

In [ ]:
#Observations count on country wise
countrywise=dataset[['Country','CustomerID']].drop_duplicates()
countrywise.groupby(['Country'])['CustomerID'].aggregate('count').reset_index().sort_values('CustomerID', ascending=False)


In [ ]:
#Conculsion - More than 90% data are from UK
#Analze the numerical feature
dataset.describe(percentiles=[0.1, 0.25, 0.5, 0.75, 0.9])



In [ ]:
dataset[(dataset['Quantity']<0)].head(5)


> Observations:
a)The stock code values aren't only numerical
b)The InvoiceNo aren't also only numerical since there is a C before the other numbers for every negative value in the quantity column, this could mean that the order was canceled.


In [ ]:
#Analyzing Invoice
temp = dataset.groupby(by=['CustomerID', 'InvoiceNo'], as_index=False)['InvoiceDate'].count()
nb_products_per_basket = temp.rename(columns = {'InvoiceDate':'Number of products'})
nb_products_per_basket.InvoiceNo = nb_products_per_basket.InvoiceNo.astype(str)
nb_products_per_basket['order_canceled'] = nb_products_per_basket['InvoiceNo'].apply(lambda x:int('C' in x))
len(nb_products_per_basket[nb_products_per_basket['order_canceled']==1])/len(nb_products_per_basket)*100


Looks like 16% of the transactions were canceled

In [ ]:
#Few customers who cancelled the products. Same customers perfomed the cancellation repeatedly 
nb_products_per_basket[nb_products_per_basket['order_canceled']==1][:5]


In [ ]:
#Lets analyze the Stockcode
list_special_codes = dataset[dataset['StockCode'].str.contains('^[a-zA-Z]+', regex=True)]['StockCode'].unique()
list_special_codes


These are specific operations which doesn't characterize our customers so I'll just drop these transactions from our database
Dropping incorrect transactions

In [ ]:
dataset = dataset[dataset['StockCode']!= 'POST']
dataset = dataset[dataset['StockCode']!= 'D']
dataset = dataset[dataset['StockCode']!= 'C2']
dataset = dataset[dataset['StockCode']!= 'M']
dataset = dataset[dataset['StockCode']!= 'BANK CHARGES']
dataset = dataset[dataset['StockCode']!= 'PADS']
dataset = dataset[dataset['StockCode']!= 'DOT']


In [ ]:
#Outliers
dataset.describe()


In [ ]:
#unit price contains the min value of 0, Sample of them


In [ ]:
dataset[(dataset['UnitPrice'] == 0)].head(5)


In [ ]:
#Feature Engineering

In [ ]:
def unique_counts(data):
   for i in data.columns:
       count = data[i].nunique()
       print(i, ": ", count)
unique_counts(dataset)


In [ ]:
import matplotlib.pyplot as plt
#No of invoces per country
No_invoice_per_country = dataset.groupby(["Country"])["InvoiceNo"].count().sort_values()
No_invoice_per_country.plot(kind='barh', figsize=(15,12))
plt.title("Number of Invoices per Country")


In [ ]:
#Label Encoding - Country
le = LabelEncoder()
le.fit(dataset['Country'])


In [ ]:
l = [i for i in range(37)]
dict(zip(list(le.classes_), l))



In [ ]:
dataset['Country'] = le.transform(dataset['Country'])

with open('labelencoder.pickle', 'wb') as g:
    pickle.dump(le, g)


In [ ]:
dataset.head(2)

In [ ]:
#RFM Analysis
dataset['InvoiceDate'].min()
dataset['InvoiceDate'].max()


In [ ]:
# I'll just fix the date to be one day after the last entry in the databse

NOW = dt.datetime(2011,12,10)
dataset['InvoiceDate'] = pd.to_datetime(dataset['InvoiceDate'])


In [ ]:
custom_aggregation = {}
custom_aggregation["InvoiceDate"] = lambda x:x.iloc[0]
custom_aggregation["CustomerID"] = lambda x:x.iloc[0]


rfmTable = dataset.groupby("InvoiceNo").agg(custom_aggregation)


In [ ]:
rfmTable["Recency"] = NOW - rfmTable["InvoiceDate"]
rfmTable["Recency"] = pd.to_timedelta(rfmTable["Recency"]).astype("timedelta64[D]")


In [ ]:
custom_aggregation = {}

custom_aggregation["Recency"] = ["min", "max"]
custom_aggregation["InvoiceDate"] = lambda x: len(x)

rfmTable_final = rfmTable.groupby("CustomerID").agg(custom_aggregation)


In [ ]:
rfmTable_final.columns = ["min_recency", "max_recency", "frequency"]


In [ ]:

rfmTable_final.head(5)




In [ ]:

quantiles = rfmTable_final.quantile(q=[0.25,0.5,0.75])
quantiles = quantiles.to_dict()


In [ ]:
segmented_rfm = rfmTable_final


In [ ]:
def RScore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
    
def FMScore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1



In [ ]:
segmented_rfm['r_quartile'] = segmented_rfm['min_recency'].apply(RScore, args=('min_recency',quantiles,))
segmented_rfm['f_quartile'] = segmented_rfm['frequency'].apply(FMScore, args=('frequency',quantiles,))
segmented_rfm.head()



In [ ]:
segmented_rfm['RFMScore'] = segmented_rfm.r_quartile.map(str) + segmented_rfm.f_quartile.map(str)
segmented_rfm.head()


In [ ]:
segmented_rfm.head(5)


In [ ]:
segmented_rfm = segmented_rfm.reset_index()


In [ ]:
segmented_rfm.head(5)


In [ ]:
df_cleaned = pd.merge(dataset,segmented_rfm, on='CustomerID')
df_cleaned.columns



In [ ]:
df_cleaned = df_cleaned.drop(columns=['r_quartile', 'f_quartile'])


In [ ]:
#Time Features

In [ ]:
df_cleaned['Month'] = df_cleaned["InvoiceDate"].map(lambda x: x.month)


In [ ]:
df_cleaned['Month'].value_counts()


In [ ]:
df_cleaned['Weekday'] = df_cleaned["InvoiceDate"].map(lambda x: x.weekday())
df_cleaned['Day'] = df_cleaned["InvoiceDate"].map(lambda x: x.day)
df_cleaned['Hour'] = df_cleaned["InvoiceDate"].map(lambda x: x.hour)


In [ ]:
df_cleaned.head(5)


In [ ]:
#Product Categories

In [ ]:
X = df_cleaned["Description"].unique()

stemmer = nltk.stem.porter.PorterStemmer()
stopword = nltk.corpus.stopwords.words('english')

def stem_and_filter(doc):
    tokens = [stemmer.stem(w) for w in analyzer(doc)]
    return [token for token in tokens if token.isalpha()]

analyzer = TfidfVectorizer().build_analyzer()
CV = TfidfVectorizer(lowercase=True, stop_words="english", analyzer=stem_and_filter, min_df=0.00, max_df=0.3)  # we remove words if it appears in more than 30 % of the corpus (not found stopwords like Box, Christmas and so on)
TF_IDF_matrix = CV.fit_transform(X)
print("TF_IDF_matrix :", TF_IDF_matrix.shape, "of", TF_IDF_matrix.dtype)




In [ ]:
svd = TruncatedSVD(n_components = 100)
normalizer = Normalizer(copy=False)

TF_IDF_embedded = svd.fit_transform(TF_IDF_matrix)
TF_IDF_embedded = normalizer.fit_transform(TF_IDF_embedded)
print("TF_IDF_embedded :", TF_IDF_embedded.shape, "of", TF_IDF_embedded.dtype)


In [ ]:
score_tfidf = []

x = list(range(5, 155, 10))

for n_clusters in x:
    kmeans = KMeans(init='k-means++', n_clusters = n_clusters, n_init=10)
    kmeans.fit(TF_IDF_embedded)
    clusters = kmeans.predict(TF_IDF_embedded)
    silhouette_avg = silhouette_score(TF_IDF_embedded, clusters)

    rep = np.histogram(clusters, bins = n_clusters-1)[0]
    score_tfidf.append(silhouette_avg)


In [ ]:
plt.figure(figsize=(20,16))

plt.subplot(2, 1, 1)
plt.plot(x, score_tfidf, label="TF-IDF matrix")
plt.title("Evolution of the Silhouette Score")
plt.legend()


In [ ]:
#The highest value for the silhouette score is when there are 135 clusters. So we'll chose this value.


In [ ]:
n_clusters = 135

kmeans = KMeans(init='k-means++', n_clusters = n_clusters, n_init=30, random_state=0)
proj = kmeans.fit_transform(TF_IDF_embedded)
clusters = kmeans.predict(TF_IDF_embedded)
plt.figure(figsize=(10,10))
plt.scatter(proj[:,0], proj[:,1], c=clusters)
plt.title("ACP with 135 clusters", fontsize="20")


In [ ]:
tsne = TSNE(n_components=2)
proj = tsne.fit_transform(TF_IDF_embedded)

plt.figure(figsize=(10,10))
plt.scatter(proj[:,0], proj[:,1], c=clusters)
plt.title("Visualization of the clustering with TSNE", fontsize="20")


In [ ]:
plt.figure(figsize=(20,8))
wc = WordCloud()

for num, cluster in enumerate(random.sample(range(100), 12)) :
    plt.subplot(3, 4, num+1)
    wc.generate(" ".join(X[np.where(clusters==cluster)]))
    plt.imshow(wc, interpolation='bilinear')
    plt.title("Cluster {}".format(cluster))
    plt.axis("off")
plt.figure()


In [ ]:
pd.Series(clusters).hist(bins=100)


dict_article_to_cluster = {article : cluster for article, cluster in zip(X, clusters)}


In [ ]:
with open('product_clusters.pickle', 'wb') as h:
    pickle.dump(dict_article_to_cluster, h)
